In [1]:
import itertools
import collections
from collections import deque
from collections import namedtuple

import sys, os, cv2, random
from time import time
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
""" More essential libraries for the task """
from gym_unity.envs import UnityEnv
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
#Importing Keras esssentials
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D

Using TensorFlow backend.


In [3]:
"""
Just some fancy folder names given to remember the various versions of the environment: NewSETAgent_<terrain_size>_<target_random/fixed>_<other_updates>\ModifiedFinalSEt
"""
env = UnityEnv("NewSETAgent_300_RandomTarget_NewReward\ModifiedFinalSEt", use_visual=True, uint8_visual=True)
print(str(env))

INFO:mlagents_envs:Connected new brain:
CarBrain?team=0
INFO:gym_unity:1 agents within environment.
<UnityEnv instance>


In [4]:
""" Code to test if the environment is running. Taken form the original documentation of Unity ML-Agents ver 0.15 """

# for episode in range(1):
#     initial_observation = env.reset()
#     done = False
#     episode_rewards = 0
#     step = 0
#     while step < 10:
#         observation, reward, done, info = env.step(env.action_space.sample())
#         plt.imshow(observation[:, :, 0])
#         #print(observation)
#         print(step," ",reward)
#         step += 1
#         episode_rewards += reward
#     print("Total reward this episode: {}".format(episode_rewards))

# initial_observation = env.reset()
# print(initial_observation.reshape(16384))
# observation_examples = np.array([env.observation_space.sample().reshape(16384) for x in range(10000)])
# print(observation_examples)

' Code to test if the environment is running. Taken form the original documentation of Unity ML-Agents ver 0.15 '

In [5]:
tf.compat.v1.reset_default_graph()
lr_actor = 1e-3
lr_critic = 1e-2
gamma_ = 0.95
frame = 0
num_episodes = 1000
load_model = False
#episode = 0

In [6]:
input_dim = env.observation_space.shape
print(input_dim)
print(env.action_space.shape)
print(env.action_space.low)
print(env.action_space.high)

""" Setting the input tensor """
visible = Input(shape=input_dim)

(64, 64, 1)
(2,)
[-1. -1.]
[1. 1.]


In [7]:
"""
To be executed the first time after installing Graphviz. Ignore, if already done. Graphviz is used to output the image of the network models used for the actor and critic agent.
"""
# os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

'\nTo be executed the first time after installing Graphviz. Ignore, if already done. Graphviz is used to output the image of the network models used for the actor and critic agent.\n'

In [8]:
""" Actor Network """
with tf.compat.v1.variable_scope("ActorNetwork", reuse = tf.compat.v1.AUTO_REUSE):

    ac_conv1 = Conv2D(32, kernel_size=3, activation='relu')(visible)
    ac_pool1 = MaxPooling2D(pool_size=(3, 3))(ac_conv1)
    ac_conv2 = Conv2D(32, kernel_size=3, activation='relu')(ac_pool1)
    ac_pool2 = MaxPooling2D(pool_size=(3, 3))(ac_conv2)
    ac_flat = Flatten()(ac_pool2)
    ac_hidden1 = Dense(20, activation='relu')(ac_flat)
    ac_hidden2 = Dense(20, activation='relu')(ac_hidden1)
    mu_output = Dense(2, activation='softmax')(ac_hidden2)
    sigma_output = Dense(2, activation = 'softmax')(ac_hidden2)
    
    normal_dist = tf.compat.v1.distributions.Normal(loc = mu_output, scale=sigma_output)
    act_out = tf.keras.backend.reshape(normal_dist.sample(1), shape=[-1,2])
    act_out = tf.clip_by_value(act_out, clip_value_min = env.action_space.low, clip_value_max = env.action_space.high)
    
    actor_model = Model(inputs=visible, outputs=[mu_output, sigma_output])
    #PLotting the model
    plot_model(actor_model, to_file='ActorNetwork_CNN.png')
    #Model summary
    actor_model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (N

In [9]:
""" Critic Network """
with tf.compat.v1.variable_scope("CriticNetwork", reuse = tf.compat.v1.AUTO_REUSE):
    ct_conv1 = Conv2D(32, kernel_size=3, activation='relu')(visible)
    ct_pool1 = MaxPooling2D(pool_size=(3, 3))(ct_conv1)
    ct_conv2 = Conv2D(32, kernel_size=3, activation='relu')(ct_pool1)
    ct_pool2 = MaxPooling2D(pool_size=(3, 3))(ct_conv2)
    ct_flat = Flatten()(ct_pool2)
    ct_hidden1 = Dense(20, activation='relu')(ct_flat)
    ct_hidden2 = Dense(20, activation='relu')(ct_hidden1)
    ct_output = Dense(2, activation='softmax')(ct_hidden2)
    critic_model = Model(inputs=visible, outputs=ct_output)
    #PLotting the model
    plot_model(critic_model, to_file='CriticNetwork_CNN.png')
    #Model Summary
    critic_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 1)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 62, 62, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 18, 18, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1152)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 20)                2306

In [10]:
actions_ = tf.compat.v1.placeholder(tf.float32, [None, 2])
returns_ = tf.compat.v1.placeholder(tf.float32, [None, 1])
logprobs = normal_dist.log_prob(actions_)

In [11]:
entropy = normal_dist.entropy()
advantages = returns_ - ct_output

In [12]:
policy_loss = tf.reduce_mean(-logprobs * advantages - 0.01*entropy)
value_loss = tf.reduce_mean(tf.square(returns_ - ct_output))

In [13]:
optimizer_policy = tf.compat.v1.train.AdamOptimizer(learning_rate=lr_actor)
optimizer_value = tf.compat.v1.train.AdamOptimizer(learning_rate=lr_critic)

In [14]:
train_policy = optimizer_policy.minimize( policy_loss,var_list=tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.GLOBAL_VARIABLES, "ActorNetwork" ))

In [15]:
train_value = optimizer_value.minimize( value_loss,var_list=tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.GLOBAL_VARIABLES, "CriticNetwork" ))

In [16]:
config = tf.compat.v1.ConfigProto()

In [17]:
sess = tf.compat.v1.Session(config=config)

In [18]:
init = tf.compat.v1.global_variables_initializer()
sess.run(init)

In [19]:
""" Execute it to load saved weights """
if load_model:
    actor_model.load_weights("./a2c_actor.h5")
    critic_model.load_weights("./a2c_critic.h5")
    print("Saved model loaded...\nStart Training.")
else:
    print("Not loading any model for now!")

Not loading any model for now!


In [20]:
writer = tf.compat.v1.summary.FileWriter("./tensorboard2/")
tf.compat.v1.summary.scalar("Policy_Loss", policy_loss)
tf.compat.v1.summary.scalar("Value_Loss", value_loss)
write_op = tf.compat.v1.summary.merge_all()

In [23]:
""" Ultimate code to train the model """
for i_ep in range(11, num_episodes):
    states_list = []
    actions_list = []
    rewards_list = []
    mu_list = []
    sigma_list = []
    ep_reward = 0
    ep_rewards = []
    step = 0
    curr_frame = env.reset()
    done = False
    curr_time = time()
    while True:
        curr_state = np.array(curr_frame.reshape(-1, 64, 64, 1))
#       curr_state = (curr_state - env.observation_space.low) / (env.observation_space.high - env.observation_space.low)
#       curr_state = tf.convert_to_tensor(curr_state, dtype = tf.float32)
#       print(curr_state.shape)
#       print(curr_state.dtype)
#       curr_state = curr_state.astype(np.float32)
#       curr_state = np.float_(curr_state)
#       print(curr_state.dtype)
#       print(states_.shape)
#       tf.image.convert_image_dtype(curr_state, dtype = tf.float32)

        mu, sigma, action = sess.run([mu_output, sigma_output, act_out], feed_dict={visible: curr_state})
        next_frame, reward, done, _ = env.step(action)
        ep_reward += reward
        curr_frame = next_frame

        reward_t = reward
        #if step % 100 == 0:
            #print("Episode:  "+str(i_ep)+" Done: "+str(done)+" Step:  "+str(step)+" Reward: "+str(reward)+" Cumulative Reward: "+str(ep_reward)+" Action: "+str(action))
        step += 1
        states_list.append(curr_state)
        actions_list.append(action)
        rewards_list.append(reward_t)
        
        mu_list.append(mu.reshape(-1,))
        sigma_list.append(sigma.reshape(-1,))
        #vprint("Done till here, Done: ", done)
        """ Ending an episode after 1000 steps are taken. Figured out 500 - 600 steps are enough to reach the target. However, if you increase the tea=rrain, the value might change. So its better to remove the restriction in most cases."""
        # if step == 1000:
        #     done = True

        if done:
            #vprint("Episode:  "+str(i_ep)+" Done: "+str(done)+" Step:  "+str(step)+" Reward: "+str(reward)+" Cumulative Reward: "+str(ep_reward)+" Action: "+str(action))
            states = np.vstack(states_list)
            actions = np.vstack(actions_list)
            rewards = np.hstack(rewards_list)
            mus = np.hstack(mu_list)
            sigmas = np.hstack(sigma_list)

            returns = np.zeros_like(rewards)
            rolling = 0
            for i in reversed(range(len(rewards))):
                rolling = rolling * gamma_ + rewards[i]
                returns[i] = rolling
            returns -= np.mean(returns)
            returns /= np.std(returns)

            feed_dict = {visible: states, actions_: actions, returns_: returns.reshape(-1, 1)}

            sess.run(train_value, feed_dict=feed_dict)
            sess.run(train_policy, feed_dict=feed_dict)
            
            summary = sess.run(write_op, feed_dict=feed_dict)
            writer.add_summary(summary, i_ep)
            writer.flush()

            print("\nEpisode : %s," % (i_ep) + \
                   "\nMean mu : %.5f, Min mu : %.5f, Max mu : %.5f, Median mu : %.5f" % \
                       (np.nanmean(mus), np.min(mus), np.max(mus), np.median(mus)) + \
                   "\nMean sigma : %.5f, Min sigma : %.5f, Max sigma : %.5f, Median sigma : %.5f" % \
                       (np.nanmean(sigmas), np.min(sigmas), np.max(sigmas), np.median(sigmas)) + \
                   "\nScores : %.4f, Max reward : %.4f, Min reward : %.4f" % (ep_reward, np.max(rewards), np.min(rewards)))
            
            actor_model.save_weights("./a2c_actor.h5")
            critic_model.save_weights("./a2c_critic.h5")
            print("Model saved.")
            break

 nan, Median sigma : nan
Scores : -0.6000, Max reward : 0.0000, Min reward : -0.6000
Model saved.

Episode : 73,
Mean mu : nan, Min mu : nan, Max mu : nan, Median mu : nan
Mean sigma : nan, Min sigma : nan, Max sigma : nan, Median sigma : nan
Scores : -0.6000, Max reward : 0.0000, Min reward : -0.6000
Model saved.

Episode : 74,
Mean mu : nan, Min mu : nan, Max mu : nan, Median mu : nan
Mean sigma : nan, Min sigma : nan, Max sigma : nan, Median sigma : nan
Scores : -0.6000, Max reward : 0.0000, Min reward : -0.6000
Model saved.

Episode : 75,
Mean mu : nan, Min mu : nan, Max mu : nan, Median mu : nan
Mean sigma : nan, Min sigma : nan, Max sigma : nan, Median sigma : nan
Scores : -0.6000, Max reward : 0.0000, Min reward : -0.6000
Model saved.

Episode : 76,
Mean mu : nan, Min mu : nan, Max mu : nan, Median mu : nan
Mean sigma : nan, Min sigma : nan, Max sigma : nan, Median sigma : nan
Scores : -0.6000, Max reward : 0.0000, Min reward : -0.6000
Model saved.

Episode : 77,
Mean mu : nan, 

KeyboardInterrupt: 

In [24]:
"Very Essential. Please close the environemt. PLEASE"
env.close()


INFO:mlagents_envs:Environment shut down with return code 0 (CTRL_C_EVENT).
